## Building Agents with RAG and Tools

### Setting up LLM & Loading the ENV file

In [1]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv(override=True)
load_dotenv('./../.env')

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.5,
    max_tokens=400
)

### Setting up PlayWright

In [2]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
import nest_asyncio

nest_asyncio.apply()

In [ ]:
async_chrome = create_async_playwright_browser()

toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_chrome)
tools_list = toolkit.get_tools()

### Setting up VectorDB & Retriever

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma(
    persist_directory='./../chroma_db_hf', 
    embedding_function=embedding_model
)

result = vector_store.similarity_search("Which country is Parveen travelling ?", k=5)

for doc in result:
    print(doc.metadata["source"])

### Setting up Retriever for the VectorDB

In [15]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs = {'k': 5})

# response = retriever.invoke("Full name of Dhanesh?")

# for doc in response:
#     print(doc.metadata["source"])


### Creating a tool to Retrieve from the VectorDB

In [16]:
from langchain.tools import tool

@tool
def retrieve_person_detail(query: str) -> str:
    """
    The tool provides employment details related to DHANESH and PARVEEN where they accepted the job offers.
    You should use this tool to check any employment information related to Dhanesh and Parveen.
    
    Args:
        query: The search query with proper detail
    """
    
    return retriever.invoke(query)

### Addition to the ToolKit

In [17]:
tools_list.append(retrieve_person_detail)

tools_list

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/par

### Creating Agent code to call the Tools

In [ ]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    tools=tools_list,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Following URL lists some information of DHANESH - https://www.weekday.works/people/dhanesh-gujrathi-dhanesh-gujrathi-9129ab55. Can you check if Dhanesh joined the same organisation in 2019 which applied for his Employment pass that year ?"

response = await agent.ainvoke(query)

response